In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df = pd.read_csv('../DagsHub/data/CrossValidated-Questions.csv')
df.head()

,Id,Title,Body,Tags,CreationDate,Score,ViewCount,AnswerCount,CommentCount,FavoriteCount
0,495508,Accepting null hypothesis given specificity,"<p>In a population of 10000 people, there is a...",<bayesian><p-value>,2020-11-08 03:59:02,0,5,0,2,NaN
1,495507,"How do I calculate the probability of victory,...",<p>If I have a poll of 600 people (standard de...,<probability><normal-distribution><t-distribut...,2020-11-08 02:09:11,0,20,0,0,NaN
2,495506,Is joint modelling with survival and longitudi...,<p>I would like to know if joint models for su...,<survival><panel-data>,2020-11-08 01:55:08,0,5,0,0,NaN
3,495503,Are Spiking Neural Networks The Next Big Thing?,<p>Intel recently announced their Loihi chip a...,<machine-learning><neural-networks><sparse><ar...,2020-11-08 00:37:13,0,18,0,4,NaN
4,495502,Public data and examples for practicing distri...,<p>Are there public data for practicing distri...,<references><learning>,2020-11-08 00:09:25,1,10,0,3,1.0


In [4]:
# preprocessing

In [5]:
df['CreationDate'] = pd.to_datetime(df['CreationDate'])
# Converting creation date to an int
df['CreationDate_Epoch'] = df['CreationDate'].astype('int64') // 10 ** 9

In [6]:
# We use tags with Ml only
df['MachineLearning'] = df['Tags'].str.contains('machine-learning').fillna(False)

In [7]:
# drop Unecessary columns 
df = df.drop(columns=['Id','Tags'])

In [8]:
df.head()

,Title,Body,CreationDate,Score,ViewCount,AnswerCount,CommentCount,FavoriteCount,CreationDate_Epoch,MachineLearning
0,Accepting null hypothesis given specificity,"<p>In a population of 10000 people, there is a...",2020-11-08 03:59:02,0,5,0,2,NaN,1604807942,False
1,"How do I calculate the probability of victory,...",<p>If I have a poll of 600 people (standard de...,2020-11-08 02:09:11,0,20,0,0,NaN,1604801351,False
2,Is joint modelling with survival and longitudi...,<p>I would like to know if joint models for su...,2020-11-08 01:55:08,0,5,0,0,NaN,1604800508,False
3,Are Spiking Neural Networks The Next Big Thing?,<p>Intel recently announced their Loihi chip a...,2020-11-08 00:37:13,0,18,0,4,NaN,1604795833,True
4,Public data and examples for practicing distri...,<p>Are there public data for practicing distri...,2020-11-08 00:09:25,1,10,0,3,1.0,1604794165,False


In [9]:
df[df['MachineLearning']!= True].sample(5)

,Title,Body,CreationDate,Score,ViewCount,AnswerCount,CommentCount,FavoriteCount,CreationDate_Epoch,MachineLearning
6267,Reversing order of outcome variable - multinom...,<p>Here is a working example of my issue:</p>\...,2020-08-15 06:11:30,2,12,0,0,NaN,1597471890,False
43475,How do GANs stay in sync?,<p>What further research has been done since t...,2019-02-11 02:56:05,1,19,0,0,NaN,1549853765,False
41337,Understanding the example in ?prcomp (R),"<p>I'm trying to understand, in simple terms, ...",2019-03-16 00:27:28,0,168,1,6,1.0,1552696048,False
12313,"Why do we need to model max, min, and average ...","<p>For COVID-19, I read many papers who try to...",2020-06-01 09:39:23,1,13,0,15,NaN,1591004363,False
23875,Regression F and t statistic,"<p>My understanding was that, to make a hypoth...",2020-01-19 20:31:36,1,30,0,2,0.0,1579465896,False


In [13]:
df[df['MachineLearning']].sample(5)

,Title,Body,CreationDate,Score,ViewCount,AnswerCount,CommentCount,FavoriteCount,CreationDate_Epoch,MachineLearning
6246,Providing latitude and longitude to a house pr...,"<p>I'm new to machine learning, and I'm trying...",2020-08-15 16:49:59,4,313,1,3,NaN,1597510199,True
41541,Apply K-means to the columns of the covariance...,"<p>In Section 5.3 of the paper <a href=""https:...",2019-03-13 06:30:14,3,203,1,1,NaN,1552458614,True
48300,Is ID column in a regression bad?,<p>Is it always a bad idea to have something l...,2018-11-14 14:01:39,0,775,1,4,NaN,1542204099,True
45792,Translation invariance of features in convolut...,<p>A big part of using convolutional layers is...,2019-01-02 11:48:06,1,179,0,0,NaN,1546429686,True
24637,What is the definition of machine learning (vs...,<p>I'm in the process of writing about the dif...,2020-01-09 02:01:52,8,1027,4,2,2.0,1578535312,True


In [14]:
df['Title_len'] = df.Title.str.len()
df['Body_len'] =df.Body.str.len()

In [15]:
# Data Exploration 

In [16]:
df.describe()

,Score,ViewCount,AnswerCount,CommentCount,FavoriteCount,CreationDate_Epoch,Title_len,Body_len
count,50000.000000,50000.000000,50000.00000,50000.000000,8918.000000,5.000000e+04,50000.000000,50000.00000
mean,1.121240,154.553100,0.67778,1.782800,1.366450,1.575066e+09,63.073700,1336.45880
std,2.313259,616.019487,0.80392,2.551077,2.199443,1.878285e+07,25.075001,1439.36074
min,-14.000000,1.000000,0.00000,0.000000,0.000000,1.539279e+09,15.000000,38.00000
25%,0.000000,23.000000,0.00000,0.000000,1.000000,1.558326e+09,45.000000,590.00000
50%,1.000000,42.000000,1.00000,1.000000,1.000000,1.578066e+09,59.000000,975.00000
75%,1.000000,97.000000,1.00000,3.000000,1.000000,1.590782e+09,78.000000,1623.00000
max,87.000000,48969.000000,32.00000,34.000000,82.000000,1.604808e+09,150.000000,29616.00000
